In [1]:
from dotenv import load_dotenv
from langchain_ollama import OllamaLLM
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableSequence,RunnableLambda

load_dotenv()
model = OllamaLLM(model="gpt-oss:20b")

In [4]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","是一個專業的文字摘要助手，擅長將長篇文章濃縮成簡潔的重點摘要。"),
    ("human","請將以下文章摘要成 {max_length} 字以內的重點：\n\n{article}")
])

In [ ]:
# 手動建立可執行的步驟(runnable)
# 這些步驟對應LCEL鍊中的各個元件

# 步驟1. 格式化提示
format_prompt = RunnableLambda(lambda x:prompt_template.from_messages(**x))

# 步驟2. 呼叫模型

RunnableLambda(lambda x:model.invoke(x))